In [1]:
import pandas as pd

In [12]:
final_df=pd.read_csv('final.csv').iloc[:,3:]

clean_test=pd.read_csv('cleaned.csv')
final_df.shape,clean_test.shape

((4991, 10), (1707, 3))

In [16]:
final_df.head()

,id,text,anger,fear,joy,sadness,surprise,emotions,clean_text,final_text
0,0,the dentist that did the work apparently did a...,1,0,0,1,0,['anger' 'sadness'],the dentist that did the work apparently did a...,dentist work apparently lousy job year teeth d...
1,1,i'm gonna absolutely ~~suck~~ be terrible duri...,0,1,0,1,0,['fear' 'sadness'],i m gonna absolutely suck be terrible during m...,gon na absolutely suck terrible first sexual e...
2,2,"bridge: so leave me drowning calling houston, ...",0,1,0,1,0,['fear' 'sadness'],"bridge: so leave me drowning calling houston, ...","bridge : leave drowning calling houston , let ..."
3,3,after that mess i went to see my now ex-girlfr...,1,1,0,1,0,['anger' 'fear' 'sadness'],after that mess i went to see my now ex girlfr...,mess went see ex girlfriend school refused dri...
4,4,"as he stumbled i ran off, afraid it might some...",0,1,0,0,0,['fear'],"as he stumbled i ran off, afraid it might some...","stumbled ran , afraid might somehow affect job..."


In [7]:
!pip install wandb

In [8]:
import wandb

In [ ]:
wandb.init()

In [15]:
wandb.init(
    project="23f3003030-t32025",
    name="Day01_EmotionModel_Baselines",
    config={
        "description": "Baseline comparison of multiple classifiers on emotion labels",
        "tfidf_max_features": 5000,
        "vectorizer_type": "TfidfVectorizer (1,2)",
        "split_ratio": 0.8,
        "experiment_type": "Multi-label classification"
    }
)


In [10]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.5 MB/s eta 0:00:00


In [20]:
"""
🚀 Industry-Grade Multi-Label Text Classification Pipeline
Features: Advanced Experiment Tracking, Comprehensive Metrics, Production-Ready Visualizations
"""

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import wandb
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Any
import json

from sklearn.exceptions import UndefinedMetricWarning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    f1_score, classification_report, confusion_matrix,
    roc_curve, auc, roc_auc_score, precision_recall_curve,
    average_precision_score, hamming_loss, jaccard_score,
    multilabel_confusion_matrix, accuracy_score
)
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# =========================
# 🎨 Configuration & Setup
# =========================
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Set style for professional visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Create output directories
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)
(OUTPUT_DIR / "plots").mkdir(exist_ok=True)
(OUTPUT_DIR / "reports").mkdir(exist_ok=True)

# Experiment configuration
EXPERIMENT_CONFIG = {
    "test_size": 0.2,
    "random_state": 42,
    "tfidf_max_features": 5000,
    "ngram_range": (1, 2),
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S")
}

# Initialize W&B with rich config
wandb.init(
    project="23f3003030-t32025",
    name=f"multi-label-classification-{EXPERIMENT_CONFIG['timestamp']}",
    config=EXPERIMENT_CONFIG,
    tags=["multi-label", "emotion-detection", "production"],
    notes="Industry-grade experiment with comprehensive tracking"
)

# =========================
# 📊 Advanced Visualization Functions
# =========================

def plot_combined_confusion_matrix(y_true, y_pred, emotions, model_name):
    """Create professional confusion matrix visualization"""
    n_emotions = len(emotions)
    fig, axes = plt.subplots(1, n_emotions, figsize=(4*n_emotions, 3.5))

    if n_emotions == 1:
        axes = [axes]

    for i, emotion in enumerate(emotions):
        cm = confusion_matrix(y_true.iloc[:, i], y_pred[:, i])

        # Calculate percentages
        cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

        # Create annotations with counts and percentages
        annot = np.array([[f'{count}\n({percent:.1f}%)'
                          for count, percent in zip(row_counts, row_percents)]
                         for row_counts, row_percents in zip(cm, cm_percent)])

        sns.heatmap(cm, annot=annot, fmt='', cmap='Blues', ax=axes[i],
                    cbar=False, square=True, linewidths=1, linecolor='gray')
        axes[i].set_title(f'{emotion.upper()}', fontsize=12, fontweight='bold', pad=10)
        axes[i].set_xlabel('Predicted', fontsize=10)
        axes[i].set_ylabel('Actual' if i == 0 else '', fontsize=10)
        axes[i].set_xticklabels(['No', 'Yes'])
        axes[i].set_yticklabels(['No', 'Yes'])

    fig.suptitle(f'{model_name} - Confusion Matrices (All Emotions)',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()

    filename = OUTPUT_DIR / "plots" / f'{model_name}_confusion_matrix.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
    wandb.log({f"{model_name}/confusion_matrix": wandb.Image(str(filename))})
    plt.close()


def plot_roc_curves(y_true, y_pred_proba, emotions, model_name):
    """Plot ROC curves with AUC scores"""
    fig, ax = plt.subplots(figsize=(12, 9))
    colors = plt.cm.Set2(np.linspace(0, 1, len(emotions)))

    roc_auc_scores = {}

    for i, (emotion, color) in enumerate(zip(emotions, colors)):
        fpr, tpr, _ = roc_curve(y_true.iloc[:, i], y_pred_proba[:, i])
        roc_auc = auc(fpr, tpr)
        roc_auc_scores[emotion] = roc_auc

        ax.plot(fpr, tpr, color=color, lw=3,
                label=f'{emotion.capitalize()} (AUC = {roc_auc:.4f})')

    # Add diagonal line
    ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier (AUC = 0.5)', alpha=0.6)

    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold')
    ax.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold')
    ax.set_title(f'{model_name} - ROC Curves', fontsize=18, fontweight='bold', pad=20)
    ax.legend(loc="lower right", fontsize=11, framealpha=0.9)
    ax.grid(alpha=0.4, linestyle='--')

    # Add mean AUC text box
    mean_auc = np.mean(list(roc_auc_scores.values()))
    textstr = f'Mean AUC: {mean_auc:.4f}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    ax.text(0.65, 0.15, textstr, transform=ax.transAxes, fontsize=13,
            verticalalignment='top', bbox=props, fontweight='bold')

    plt.tight_layout()
    filename = OUTPUT_DIR / "plots" / f'{model_name}_roc_curves.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
    wandb.log({f"{model_name}/roc_curves": wandb.Image(str(filename))})
    plt.close()

    return roc_auc_scores


def plot_precision_recall_curves(y_true, y_pred_proba, emotions, model_name):
    """Plot Precision-Recall curves"""
    fig, ax = plt.subplots(figsize=(12, 9))
    colors = plt.cm.Set3(np.linspace(0, 1, len(emotions)))

    avg_precision_scores = {}

    for i, (emotion, color) in enumerate(zip(emotions, colors)):
        precision, recall, _ = precision_recall_curve(y_true.iloc[:, i], y_pred_proba[:, i])
        avg_precision = average_precision_score(y_true.iloc[:, i], y_pred_proba[:, i])
        avg_precision_scores[emotion] = avg_precision

        ax.plot(recall, precision, color=color, lw=3,
                label=f'{emotion.capitalize()} (AP = {avg_precision:.4f})')

    ax.set_xlabel('Recall', fontsize=14, fontweight='bold')
    ax.set_ylabel('Precision', fontsize=14, fontweight='bold')
    ax.set_title(f'{model_name} - Precision-Recall Curves', fontsize=18, fontweight='bold', pad=20)
    ax.legend(loc="lower left", fontsize=11, framealpha=0.9)
    ax.grid(alpha=0.4, linestyle='--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])

    # Add mean AP text box
    mean_ap = np.mean(list(avg_precision_scores.values()))
    textstr = f'Mean AP: {mean_ap:.4f}'
    props = dict(boxstyle='round', facecolor='lightblue', alpha=0.8)
    ax.text(0.65, 0.95, textstr, transform=ax.transAxes, fontsize=13,
            verticalalignment='top', bbox=props, fontweight='bold')

    plt.tight_layout()
    filename = OUTPUT_DIR / "plots" / f'{model_name}_precision_recall.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
    wandb.log({f"{model_name}/precision_recall": wandb.Image(str(filename))})
    plt.close()

    return avg_precision_scores


def plot_classification_report(class_report_dict, emotions, model_name):
    """Visualize classification report as heatmap"""
    metrics = ['precision', 'recall', 'f1-score']
    data = []

    for emotion in emotions:
        if emotion in class_report_dict:
            row = [class_report_dict[emotion][m] for m in metrics]
            data.append(row)

    df_report = pd.DataFrame(data, index=[e.capitalize() for e in emotions], columns=metrics)

    fig, ax = plt.subplots(figsize=(10, 6))
    sns.heatmap(df_report, annot=True, fmt='.4f', cmap='RdYlGn',
                cbar_kws={'label': 'Score'}, vmin=0, vmax=1, ax=ax,
                linewidths=2, linecolor='white', annot_kws={"size": 12, "weight": "bold"})
    ax.set_title(f'{model_name} - Classification Metrics Heatmap',
                 fontsize=18, fontweight='bold', pad=20)
    ax.set_xlabel('Metrics', fontsize=14, fontweight='bold')
    ax.set_ylabel('Emotions', fontsize=14, fontweight='bold')

    plt.tight_layout()
    filename = OUTPUT_DIR / "plots" / f'{model_name}_classification_report.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
    wandb.log({f"{model_name}/classification_report": wandb.Image(str(filename))})
    plt.close()


def plot_per_emotion_metrics(y_true, y_pred, emotions, model_name):
    """Bar plot of per-emotion performance metrics"""
    metrics_data = {'Emotion': [], 'Precision': [], 'Recall': [], 'F1-Score': []}

    for i, emotion in enumerate(emotions):
        from sklearn.metrics import precision_recall_fscore_support
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true.iloc[:, i], y_pred[:, i], average='binary', zero_division=0
        )
        metrics_data['Emotion'].append(emotion.capitalize())
        metrics_data['Precision'].append(precision)
        metrics_data['Recall'].append(recall)
        metrics_data['F1-Score'].append(f1)

    df_metrics = pd.DataFrame(metrics_data)
    df_melted = df_metrics.melt(id_vars='Emotion', var_name='Metric', value_name='Score')

    fig, ax = plt.subplots(figsize=(12, 7))
    sns.barplot(data=df_melted, x='Emotion', y='Score', hue='Metric', ax=ax, palette='muted')

    ax.set_title(f'{model_name} - Per-Emotion Performance',
                 fontsize=18, fontweight='bold', pad=20)
    ax.set_xlabel('Emotion', fontsize=14, fontweight='bold')
    ax.set_ylabel('Score', fontsize=14, fontweight='bold')
    ax.set_ylim([0, 1.05])
    ax.legend(title='Metric', fontsize=12, title_fontsize=13)
    ax.grid(axis='y', alpha=0.4, linestyle='--')

    # Add value labels on bars
    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f', fontsize=9, padding=3)

    plt.tight_layout()
    filename = OUTPUT_DIR / "plots" / f'{model_name}_per_emotion_metrics.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
    wandb.log({f"{model_name}/per_emotion_metrics": wandb.Image(str(filename))})
    plt.close()


def plot_label_distribution(y_data, title, filename):
    """Plot label distribution"""
    label_counts = y_data.sum().sort_values(ascending=False)

    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(range(len(label_counts)), label_counts.values, color=plt.cm.viridis(np.linspace(0, 1, len(label_counts))))
    ax.set_xticks(range(len(label_counts)))
    ax.set_xticklabels([label.capitalize() for label in label_counts.index], fontsize=12)
    ax.set_ylabel('Count', fontsize=14, fontweight='bold')
    ax.set_title(title, fontsize=18, fontweight='bold', pad=20)
    ax.grid(axis='y', alpha=0.4, linestyle='--')

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}', ha='center', va='bottom', fontsize=11, fontweight='bold')

    plt.tight_layout()
    save_path = OUTPUT_DIR / "plots" / filename
    plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
    wandb.log({f"data_analysis/{filename.replace('.png', '')}": wandb.Image(str(save_path))})
    plt.close()


def compute_advanced_metrics(y_true, y_pred, y_pred_proba, emotions):
    """Compute comprehensive metrics"""
    metrics = {
        'f1_macro': f1_score(y_true, y_pred, average='macro', zero_division=0),
        'f1_micro': f1_score(y_true, y_pred, average='micro', zero_division=0),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted', zero_division=0),
        'hamming_loss': hamming_loss(y_true, y_pred),
        'jaccard_score': jaccard_score(y_true, y_pred, average='samples', zero_division=0),
        'subset_accuracy': accuracy_score(y_true, y_pred),
    }

    # Per-emotion metrics
    for i, emotion in enumerate(emotions):
        metrics[f'{emotion}_f1'] = f1_score(y_true.iloc[:, i], y_pred[:, i], zero_division=0)
        metrics[f'{emotion}_auc'] = roc_auc_score(y_true.iloc[:, i], y_pred_proba[:, i])
        metrics[f'{emotion}_ap'] = average_precision_score(y_true.iloc[:, i], y_pred_proba[:, i])

    return metrics


# =========================
# 1️⃣ Data Preparation & EDA
# =========================
print("=" * 80)
print("🚀 STARTING EXPERIMENT PIPELINE")
print("=" * 80)

X = final_df['final_text'].fillna('')
y = final_df[['anger', 'fear', 'joy', 'sadness', 'surprise']]
emotions = y.columns.tolist()

# Log dataset info
wandb.log({
    "dataset/total_samples": len(X),
    "dataset/num_emotions": len(emotions),
    "dataset/feature_name": "final_text"
})

# Analyze and log label distribution
print("\n📊 Analyzing Label Distribution...")
plot_label_distribution(y, "Training Data - Emotion Distribution", "train_label_distribution.png")

# Log label statistics
label_stats = {}
for emotion in emotions:
    label_stats[f"dataset/{emotion}_count"] = int(y[emotion].sum())
    label_stats[f"dataset/{emotion}_percentage"] = float(y[emotion].sum() / len(y) * 100)

wandb.log(label_stats)

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=EXPERIMENT_CONFIG['test_size'],
    random_state=EXPERIMENT_CONFIG['random_state']
)

print(f"✅ Train set size: {len(X_train)} | Validation set size: {len(X_val)}")

# =========================
# 2️⃣ Feature Engineering
# =========================
print("\n🔧 Extracting TF-IDF Features...")
tfidf = TfidfVectorizer(
    max_features=EXPERIMENT_CONFIG['tfidf_max_features'],
    ngram_range=EXPERIMENT_CONFIG['ngram_range']
)
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)

print(f"✅ TF-IDF shape: {X_train_tfidf.shape}")
wandb.log({"features/tfidf_shape": X_train_tfidf.shape[1]})

# =========================
# 3️⃣ Model Configuration
# =========================
classifiers = {
    'LogisticRegression': LogisticRegression(solver='liblinear', max_iter=300, random_state=42),
    'RandomForest': RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
    'LinearSVC': LinearSVC(max_iter=5000, random_state=42),
    'MultinomialNB': MultinomialNB(),
    'KNeighbors': KNeighborsClassifier(n_neighbors=8, n_jobs=-1),
    'XGBoost': XGBClassifier(
        eval_metric='logloss', n_estimators=500, max_depth=6,
        learning_rate=0.05, random_state=42, n_jobs=-1
    ),
    'LightGBM': LGBMClassifier(
        n_estimators=500, max_depth=8, learning_rate=0.05,
        random_state=42, verbose=-1, n_jobs=-1
    ),
    'CatBoost': CatBoostClassifier(
        iterations=500, depth=7, learning_rate=0.05,
        verbose=0, random_state=42, thread_count=-1
    )
}

# =========================
# 4️⃣ Model Training & Evaluation
# =========================
print("\n" + "=" * 80)
print("🤖 TRAINING AND EVALUATING MODELS")
print("=" * 80)

results = []
all_model_metrics = {}

for name, clf in classifiers.items():
    print(f"\n{'='*60}")
    print(f"🔄 Training: {name}")
    print(f"{'='*60}")

    # Train model
    model = OneVsRestClassifier(clf, n_jobs=-1)
    model.fit(X_train_tfidf, y_train)

    # Predictions
    y_pred = model.predict(X_val_tfidf)

    # Get probabilities
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_val_tfidf)
    elif hasattr(model, "decision_function"):
        y_pred_proba = model.decision_function(X_val_tfidf)
        # Normalize to [0, 1]
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        y_pred_proba = scaler.fit_transform(y_pred_proba)
    else:
        y_pred_proba = y_pred

    # ---- Compute Comprehensive Metrics ----
    metrics = compute_advanced_metrics(y_val, y_pred, y_pred_proba, emotions)
    all_model_metrics[name] = metrics

    # Log all metrics to W&B
    for metric_name, value in metrics.items():
        wandb.log({f"{name}/metrics/{metric_name}": value})

    # ---- Generate Classification Report ----
    class_report = classification_report(
        y_val, y_pred, target_names=emotions,
        output_dict=True, zero_division=0
    )

    # ---- Create All Visualizations ----
    print(f"   📊 Generating visualizations...")

    # 1. Combined Confusion Matrix
    plot_combined_confusion_matrix(y_val, y_pred, emotions, name)

    # 2. ROC Curves
    try:
        roc_scores = plot_roc_curves(y_val, y_pred_proba, emotions, name)
        wandb.log({f"{name}/metrics/mean_roc_auc": np.mean(list(roc_scores.values()))})
    except Exception as e:
        print(f"   ⚠️  Could not plot ROC curves: {e}")

    # 3. Precision-Recall Curves
    try:
        pr_scores = plot_precision_recall_curves(y_val, y_pred_proba, emotions, name)
        wandb.log({f"{name}/metrics/mean_average_precision": np.mean(list(pr_scores.values()))})
    except Exception as e:
        print(f"   ⚠️  Could not plot PR curves: {e}")

    # 4. Classification Report Heatmap
    plot_classification_report(class_report, emotions, name)

    # 5. Per-Emotion Metrics
    plot_per_emotion_metrics(y_val, y_pred, emotions, name)

    # ---- Save detailed results ----
    results.append({
        'Model': name,
        'F1_Macro': metrics['f1_macro'],
        'F1_Micro': metrics['f1_micro'],
        'F1_Weighted': metrics['f1_weighted'],
        'Hamming_Loss': metrics['hamming_loss'],
        'Jaccard_Score': metrics['jaccard_score'],
        'Subset_Accuracy': metrics['subset_accuracy']
    })

    print(f"   ✅ {name} completed!")
    print(f"      F1-Macro: {metrics['f1_macro']:.4f} | Hamming Loss: {metrics['hamming_loss']:.4f}")

# =========================
# 5️⃣ Model Comparison & Best Model Selection
# =========================
print("\n" + "=" * 80)
print("📈 MODEL COMPARISON RESULTS")
print("=" * 80)

results_df = pd.DataFrame(results).sort_values(by='F1_Macro', ascending=False)
print("\n", results_df.to_string(index=False))

# Save results to CSV
results_path = OUTPUT_DIR / "reports" / "model_comparison.csv"
results_df.to_csv(results_path, index=False)
wandb.save(str(results_path))

# Create comparison visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

metrics_to_plot = ['F1_Macro', 'F1_Micro', 'F1_Weighted', 'Hamming_Loss', 'Jaccard_Score', 'Subset_Accuracy']

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx]
    data = results_df.sort_values(by=metric, ascending=(metric == 'Hamming_Loss'))

    bars = ax.barh(data['Model'], data[metric], color=plt.cm.viridis(np.linspace(0, 1, len(data))))
    ax.set_xlabel('Score', fontsize=12, fontweight='bold')
    ax.set_title(metric.replace('_', ' '), fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.4, linestyle='--')

    # Add value labels
    for bar in bars:
        width = bar.get_width()
        ax.text(width, bar.get_y() + bar.get_height()/2.,
                f'{width:.4f}', ha='left', va='center', fontsize=10, fontweight='bold')

plt.suptitle('Model Performance Comparison - All Metrics', fontsize=20, fontweight='bold', y=1.00)
plt.tight_layout()

comparison_path = OUTPUT_DIR / "plots" / "model_comparison_all_metrics.png"
plt.savefig(comparison_path, dpi=150, bbox_inches='tight', facecolor='white')
wandb.log({"comparison/all_metrics": wandb.Image(str(comparison_path))})
plt.close()

# Log results table to W&B
wandb.log({"comparison/results_table": wandb.Table(dataframe=results_df)})

# =========================
# 6️⃣ Best Model Retraining
# =========================
best_model_name = results_df.iloc[0]['Model']
print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   F1-Macro Score: {results_df.iloc[0]['F1_Macro']:.4f}")

wandb.log({
    "best_model/name": best_model_name,
    "best_model/f1_macro": results_df.iloc[0]['F1_Macro']
})

# Retrain on full data
print(f"\n🔄 Retraining {best_model_name} on full dataset...")
best_clf = classifiers[best_model_name]
best_model = OneVsRestClassifier(best_clf, n_jobs=-1)

X_full_tfidf = tfidf.fit_transform(X)
best_model.fit(X_full_tfidf, y)

print("   ✅ Best model retrained on full dataset")

# =========================
# 7️⃣ Test Set Prediction
# =========================
print("\n📝 Generating predictions on test set...")
clean_test['final_text'] = clean_test['final_text'].fillna('')
X_test_tfidf = tfidf.transform(clean_test['final_text'])
y_test_pred = best_model.predict(X_test_tfidf)

# Create submission
submission = pd.DataFrame(y_test_pred, columns=y.columns)
submission['id'] = clean_test['id']
submission = submission[['id'] + list(y.columns)]

submission_path = OUTPUT_DIR / "submission.csv"
submission.to_csv(submission_path, index=False)
wandb.save(str(submission_path))

print(f"   ✅ Submission saved: {submission_path}")

# =========================
# 8️⃣ Final Summary Report
# =========================
print("\n" + "=" * 80)
print("📋 EXPERIMENT SUMMARY")
print("=" * 80)

summary = {
    "timestamp": EXPERIMENT_CONFIG['timestamp'],
    "best_model": best_model_name,
    "best_f1_macro": float(results_df.iloc[0]['F1_Macro']),
    "total_models_trained": len(classifiers),
    "dataset_size": len(X),
    "validation_size": len(X_val),
    "num_emotions": len(emotions),
    "emotions": emotions
}

summary_path = OUTPUT_DIR / "reports" / "experiment_summary.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=4)

print(json.dumps(summary, indent=2))
print(f"\n✅ Summary saved: {summary_path}")

wandb.log({"experiment/summary": summary})

# =========================
# 🎯 Finish Experiment
# =========================
print("\n" + "=" * 80)
print("✅ EXPERIMENT COMPLETED SUCCESSFULLY!")
print("=" * 80)
print(f"📁 All outputs saved in: {OUTPUT_DIR}")
print(f"🔗 View results in W&B: {wandb.run.get_url()}")
print("=" * 80)

wandb.finish()

LogisticRegression/metrics/anger_ap,▁
LogisticRegression/metrics/anger_auc,▁
LogisticRegression/metrics/anger_f1,▁
LogisticRegression/metrics/f1_macro,▁
LogisticRegression/metrics/f1_micro,▁
LogisticRegression/metrics/f1_weighted,▁
LogisticRegression/metrics/fear_ap,▁
LogisticRegression/metrics/fear_auc,▁
LogisticRegression/metrics/fear_f1,▁
LogisticRegression/metrics/hamming_loss,▁
+26,...


🚀 STARTING EXPERIMENT PIPELINE

📊 Analyzing Label Distribution...
✅ Train set size: 3992 | Validation set size: 999

🔧 Extracting TF-IDF Features...
✅ TF-IDF shape: (3992, 5000)

🤖 TRAINING AND EVALUATING MODELS

🔄 Training: LogisticRegression
   📊 Generating visualizations...
   ✅ LogisticRegression completed!
      F1-Macro: 0.3160 | Hamming Loss: 0.2378

🔄 Training: RandomForest
   📊 Generating visualizations...
   ✅ RandomForest completed!
      F1-Macro: 0.4584 | Hamming Loss: 0.2350

🔄 Training: LinearSVC
   📊 Generating visualizations...
   ✅ LinearSVC completed!
      F1-Macro: 0.4584 | Hamming Loss: 0.2517

🔄 Training: MultinomialNB
   📊 Generating visualizations...
   ✅ MultinomialNB completed!
      F1-Macro: 0.2706 | Hamming Loss: 0.2386

🔄 Training: KNeighbors
   📊 Generating visualizations...
   ✅ KNeighbors completed!
      F1-Macro: 0.2212 | Hamming Loss: 0.3029

🔄 Training: XGBoost
   📊 Generating visualizations...
   ✅ XGBoost completed!
      F1-Macro: 0.3907 | Hammi

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


   📊 Generating visualizations...
   ✅ LightGBM completed!
      F1-Macro: 0.3613 | Hamming Loss: 0.2589

🔄 Training: CatBoost


/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


   📊 Generating visualizations...
   ✅ CatBoost completed!
      F1-Macro: 0.4143 | Hamming Loss: 0.2352

📈 MODEL COMPARISON RESULTS

              Model  F1_Macro  F1_Micro  F1_Weighted  Hamming_Loss  Jaccard_Score  Subset_Accuracy
      RandomForest  0.458411  0.560629     0.533863      0.235035       0.402519         0.282282
         LinearSVC  0.458362  0.547027     0.533251      0.251652       0.391208         0.268268
          CatBoost  0.414270  0.536123     0.498277      0.235235       0.374741         0.277277
           XGBoost  0.390683  0.534210     0.486221      0.241241       0.378879         0.256256
          LightGBM  0.361329  0.482593     0.450200      0.258859       0.324675         0.223223
LogisticRegression  0.316025  0.505824     0.435056      0.237838       0.347931         0.256256
     MultinomialNB  0.270570  0.500838     0.398865      0.238639       0.353604         0.242242
        KNeighbors  0.221153  0.440252     0.338767      0.302903       0.292626 

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


   ✅ Submission saved: outputs/submission.csv

📋 EXPERIMENT SUMMARY
{
  "timestamp": "20251010_055437",
  "best_model": "RandomForest",
  "best_f1_macro": 0.4584105219752548,
  "total_models_trained": 8,
  "dataset_size": 4991,
  "validation_size": 999,
  "num_emotions": 5,
  "emotions": [
    "anger",
    "fear",
    "joy",
    "sadness",
    "surprise"
  ]
}

✅ Summary saved: outputs/reports/experiment_summary.json

✅ EXPERIMENT COMPLETED SUCCESSFULLY!
📁 All outputs saved in: outputs
🔗 View results in W&B: https://wandb.ai/aiwithajay-indian-institute-of-technology-madras/23f3003030-t32025/runs/d3bzjxc1


CatBoost/metrics/anger_ap,▁
CatBoost/metrics/anger_auc,▁
CatBoost/metrics/anger_f1,▁
CatBoost/metrics/f1_macro,▁
CatBoost/metrics/f1_micro,▁
CatBoost/metrics/f1_weighted,▁
CatBoost/metrics/fear_ap,▁
CatBoost/metrics/fear_auc,▁
CatBoost/metrics/fear_f1,▁
CatBoost/metrics/hamming_loss,▁
+188,...


Here’s a structured **inference report, results summary, and conclusion** based on your multi-label emotion classification experiment with 8 models:

---

## **1️⃣ Inference Report**

* **Task:** Multi-label emotion classification on 5 emotions (`anger`, `fear`, `joy`, `sadness`, `surprise`)

* **Dataset:**

  * Train: 3992 samples
  * Validation: 999 samples
  * Features: TF-IDF vectorization of text (`max_features=5000`, n-grams 1–2)

* **Models Trained:**
  Logistic Regression, Random Forest, LinearSVC, MultinomialNB, KNeighbors, XGBoost, LightGBM, CatBoost

* **Metrics Used:**

  * `F1-macro`, `F1-micro`, `F1-weighted`
  * Hamming Loss
  * Jaccard Score
  * Subset Accuracy
  * Per-class `AP`, `AUC`, `F1`

* **Hamming Loss Interpretation:**
  Fraction of labels incorrectly predicted. Lower is better.

---

## **2️⃣ Results Summary**

| Model              | F1-Macro | F1-Micro | Hamming Loss | Subset Accuracy |
| ------------------ | -------- | -------- | ------------ | --------------- |
| RandomForest       | 0.4584   | 0.5606   | 0.2350       | 0.2823          |
| LinearSVC          | 0.4584   | 0.5470   | 0.2517       | 0.2683          |
| CatBoost           | 0.4143   | 0.5361   | 0.2352       | 0.2773          |
| XGBoost            | 0.3907   | 0.5342   | 0.2412       | 0.2563          |
| LightGBM           | 0.3613   | 0.4826   | 0.2589       | 0.2232          |
| LogisticRegression | 0.3160   | 0.5058   | 0.2378       | 0.2563          |
| MultinomialNB      | 0.2706   | 0.5008   | 0.2386       | 0.2422          |
| KNeighbors         | 0.2212   | 0.4403   | 0.3029       | 0.1512          |

**Observations:**

1. **Best Overall Model:** `RandomForest`

   * Highest `F1-macro` (0.4584) and `F1-micro` (0.5606)
   * Hamming loss is among the lowest (0.2350) → good balance between false positives and false negatives
   * Subset accuracy is low (~28%), typical for multi-label tasks

2. **LinearSVC:** Almost identical macro F1 to RandomForest, slightly higher Hamming loss → slightly less reliable in individual label predictions

3. **CatBoost:** Strong F1-micro and competitive Hamming loss; slightly lower F1-macro → good at predicting frequent labels but less on rare ones

4. **Traditional Baselines (LogisticRegression, MultinomialNB, KNeighbors):**

   * Underperforming on F1-macro, especially KNeighbors (0.2212)
   * Hamming loss higher → many label prediction errors

5. **Boosting Models (XGBoost, LightGBM, CatBoost):**

   * Perform better than simpler baselines
   * CatBoost is more stable across macro/micro F1

---

## **3️⃣ Inference Highlights**

* **Rare label issue:**

  * `anger` has very low F1 (e.g., 0.018 for LogisticRegression, 0.208 for CatBoost)
  * Imbalanced label distribution is causing low per-class performance
* **Hamming Loss:**

  * Average ~0.24–0.25 for top models → ~24–25% of individual label predictions are wrong
* **F1-macro vs F1-micro:**

  * Macro gives equal weight to all labels → lower for rare labels
  * Micro favors frequent labels → top models achieve ~0.56

---

## **4️⃣ Conclusions**

1. **Best Performing Model:**

   * `RandomForest` is recommended for production use
   * Retrained on full dataset and predictions generated on the test set

2. **Multi-Label Challenge:**

   * Rare labels (`anger`, `surprise`) remain difficult to predict accurately
   * Consider strategies like **label-specific class weighting**, **oversampling**, or **focal loss**

3. **Next Steps / Improvements:**

   * Implement **ensemble of top models** (RandomForest + CatBoost + LinearSVC)
   * Experiment with **deep learning models** (BERT-based) for better text feature extraction
   * Visualize **label correlations** to exploit dependencies between emotions

4. **Submission:**

   * Generated predictions saved in `outputs/submission.csv`
   * Ready for evaluation in competition or downstream use

---

✅ **Overall:** The experiment shows that traditional tree-based models (RandomForest, CatBoost) are more effective than simple baselines in multi-label emotion classification. Rare labels remain challenging, indicating room for improvement with advanced techniques.

---

If you want, I can **also generate a single visual summary plot** combining:

* Confusion matrix of all 5 labels in **one 5x5 matrix**
* Per-class F1, precision, recall
* ROC-AUC curves

This will make the report visually much more interpretable.

Do you want me to do that next?


Absolutely! Let’s break **F1 score** in a very simple, intuitive way and explain each term.

---

## **1️⃣ What is F1 Score?**

F1 Score is the **harmonic mean of Precision and Recall**.
It balances **how many predictions were correct** with **how many true positives were found**.

[
\text{F1 Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
]

It ranges from **0 to 1**:

* **1** → perfect predictions
* **0** → worst predictions

---

## **2️⃣ Components of F1 Score**

### **a) True Positives (TP)**

* **What it means:** Correctly predicted positive examples.
* **Example:** You predicted `fear` and it was actually `fear`. ✅

### **b) False Positives (FP)**

* **What it means:** Predicted positive but it was actually negative.
* **Example:** You predicted `anger` but the true label wasn’t `anger`. ❌

### **c) False Negatives (FN)**

* **What it means:** Didn’t predict positive but it was actually positive.
* **Example:** You didn’t predict `joy` but the text actually had `joy`. ❌

---

### **3️⃣ Precision**

[
\text{Precision} = \frac{TP}{TP + FP}
]

* **Meaning:** Of all the labels you predicted as positive, how many were actually correct?
* **Focus:** **Avoiding false alarms**.
* **Example:** You predicted `fear` 100 times, but only 70 were correct → Precision = 0.7

---

### **4️⃣ Recall (Sensitivity)**

[
\text{Recall} = \frac{TP}{TP + FN}
]

* **Meaning:** Of all actual positive labels, how many did you predict correctly?
* **Focus:** **Finding all true positives**.
* **Example:** There were 80 true `fear` labels, you predicted 70 → Recall = 0.875

---

### **5️⃣ F1 Score**

* **Meaning:** Combines Precision and Recall into one number.
* **Harmonic mean** is used instead of average → punishes extreme imbalance between Precision and Recall.
* **Example:**

  * Precision = 0.7, Recall = 0.875
  * F1 = 2 * (0.7*0.875)/(0.7+0.875) ≈ **0.778**

✅ Interpretation: Good balance between avoiding false positives and capturing true positives.

---

### **6️⃣ Why F1 Score is Useful**

* If you **care about both precision and recall**, not just one → use F1.
* Especially useful in **imbalanced datasets** (like `anger` vs `fear` in your example).

---

### **Quick Intuition Example Using Your Emotion Data**

Suppose we predict `anger` (rare label) in 5 texts:

| True Labels | Predicted Labels |
| ----------- | ---------------- |
| anger       | anger            |
| fear        | anger            |
| anger       | -                |
| sadness     | -                |
| anger       | anger            |

* **TP** = 2 (`anger` correctly predicted)
* **FP** = 1 (`anger` predicted but actually `fear`)
* **FN** = 1 (`anger` was true but not predicted)

[
\text{Precision} = 2 / (2+1) = 0.667
]
[
\text{Recall} = 2 / (2+1) = 0.667
]
[
\text{F1 Score} = 2 * (0.667*0.667)/(0.667+0.667) = 0.667
]

* **Interpretation:** Your model is moderately good at detecting `anger`, balancing correct predictions and missing some.

---

If you want, I can **draw a simple visual diagram showing TP, FP, FN, Precision, Recall, and F1** for your 5 emotions dataset.

Do you want me to make that visual?


Absolutely! Let’s break down each metric in a **simple, intuitive way** using your multi-label emotion classification example. I’ll explain **what it means, why it’s important, and how it helps you understand model performance**.

---

## **1️⃣ F1-Macro**

**Definition (Easy):**

* F1-Macro calculates the F1 score **separately for each label** and then takes the **average**.
* It gives **equal importance to all labels**, whether they are frequent or rare.

**Why we use it:**

* In multi-label classification, some labels (like `anger`) are rare and some (like `fear`) are frequent.
* F1-Macro ensures the model does **well on every label**, not just the common ones.

**How it helps:**

* Tells us if the model is **fair across all emotions**, not just the ones that appear most often.

**Example (your data):**

* `fear` is common, `anger` is rare.
* LogisticRegression: F1 for `anger` = 0.018, F1 for `fear` = 0.720 → Macro F1 = 0.316
* Even though `fear` is high, Macro F1 is low because `anger` is poorly predicted.
* **Lesson:** Macro F1 highlights weaknesses on rare emotions.

---

## **2️⃣ F1-Micro**

**Definition (Easy):**

* F1-Micro calculates **global F1 score** across **all individual label predictions**.
* It treats every label prediction equally and sums over **all true positives, false positives, and false negatives**.

**Why we use it:**

* Focuses on **overall accuracy across all predictions**, giving more weight to frequent labels.

**How it helps:**

* Shows how well the model performs **overall**, regardless of label rarity.

**Example (your data):**

* LogisticRegression F1-Micro = 0.505 → 50% of all label predictions are correct on average.
* Favors `fear` (common label) over `anger` (rare label) because there are more `fear` labels.

---

## **3️⃣ Hamming Loss**

**Definition (Easy):**

* Fraction of labels **predicted incorrectly** (either missed or wrong) over all labels.
* Lower is better.

**Why we use it:**

* Multi-label classification can have **multiple correct labels per sample**.
* Hamming Loss tells you **how often the model makes mistakes on individual labels**, rather than the whole sample.

**How it helps:**

* Gives a fine-grained view of **per-label errors**.
* Useful to understand **error rate for each emotion**.

**Example (your data):**

* Hamming Loss for RandomForest = 0.235 → ~23.5% of all label predictions are wrong.
* If a text has 5 labels, 1–2 labels are predicted incorrectly on average.

---

## **4️⃣ Subset Accuracy (Exact Match Ratio)**

**Definition (Easy):**

* The percentage of samples where **all labels are predicted correctly**.
* Very strict metric.

**Why we use it:**

* Measures **complete prediction correctness**, not just individual labels.
* Important when you need **all labels to be correct** for a sample.

**How it helps:**

* Shows how often the model gets the **entire set of emotions right**.
* Very low for multi-label problems with rare labels.

**Example (your data):**

* Subset Accuracy for RandomForest = 0.282 → Only ~28% of texts had **all 5 emotions predicted exactly right**.
* Even if F1-macro or F1-micro is good, subset accuracy can be low because predicting every label correctly is hard.

---

### ✅ **Summary Table of Metrics (Intuitive)**

| Metric          | What it Measures                            | Focus                          | Example Insight                                                     |
| --------------- | ------------------------------------------- | ------------------------------ | ------------------------------------------------------------------- |
| F1-Macro        | Average F1 across all labels                | Rare & frequent labels equally | LogisticRegression F1-Macro = 0.316 → rare labels predicted poorly  |
| F1-Micro        | Global F1 across all label predictions      | Frequent labels dominate       | F1-Micro = 0.505 → overall good performance, favors frequent labels |
| Hamming Loss    | Fraction of label prediction mistakes       | Per-label error rate           | 0.235 → 23.5% of labels wrong                                       |
| Subset Accuracy | Fraction of samples with all labels correct | Exact full prediction          | 0.282 → only 28% of texts got all labels right                      |

---

💡 **Key Takeaways:**

* **Macro F1** → fairness across labels
* **Micro F1** → overall performance, dominated by common labels
* **Hamming Loss** → fine-grained per-label error
* **Subset Accuracy** → strict measure, complete correctness

---

If you want, I can **draw a visual diagram showing all 4 metrics using your RandomForest results** so you can **see clearly how each metric differs**.

Do you want me to make that visual?
